In [1]:
import os.path                      # OS処理
from collections import OrderedDict # 順序付き辞書(dict)

In [58]:
# 環境変数
root_path  = './test'            # 探索するパスの根 windows UNC path ("//host/computer/dir") を想定
search_file_name = 'keyfile.txt' # 探索するファイル名
target_file_path = './tmp'       # 保存先
headline_depth = 3               # index.rst でタイトル表示する階層数

In [3]:
# Windows のセパレータ'\\'への対応として、セパレータが '/' ではない場合は、 '/' を os.sep (OSのデフォルトセパレータ) に置き換える
if os.sep != '/':
    print('re')
    root_path = root_path.replace('/', os.sep)
    target_file_path = target_file_path.replace('/',os.sep)

In [4]:
# 指定のファイル名がリストに存在するかの判定
def isContain(filelist, keywoard):
    for filename in filelist:
        if filename == keywoard:
            return True

In [5]:
# 指定した path を巡回して、target_path_list を作る

#target_dict = OrderedDict() # 順序付き辞書(dict)
target_path_list = [] # 辞書入れリスト

for root, dirs, files in os.walk(root_path): # 相対path, サブディレクトリ, 内含ファイルリスト を走査

    #print( root,dirs,files) # debug
    
    if (isContain(files, search_file_name)): # 指定するファイル名を含むディレクトリの場合は以下を処理
        drive, path = os.path.splitdrive(root) # ネットワークドライブ名とパス名を分離         
        target_dict = {'drive': drive,                                 # windows 共有ディレクトリのドライブ名
                       'path': os.path.join(path, search_file_name),   # 元ネタファイルの相対パス
                       'name': os.path.basename(path),                 # 最終ディレクトリ名を生成対象ファイル名に
                       'depth': path.count(os.sep)}                    # 階層の深さを
        target_path_list.append(target_dict)
    

In [60]:
before_depth  = 65535
depth_count   = 1

for target in target_path_list:                       # target_path_list を順序良く評価していく
#    print(target)
#    print("before_depth =",before_depth)
#    print("current_depth=",target['depth'])
#    print("count        =",depth_count)
#    print("end_depth    =",end_depth)
    
    if target['depth'] < before_depth :               # 評価の途中で階層が浅くなった場合は評価をリセット
        depth_count = 1                               # Headline 化階層カウンタ

    if target['depth'] > before_depth :               # 評価の途中で階層が深くなった場合はカウンタ++
        depth_count = depth_count + 1

    if depth_count <= headline_depth :                # カウンタが headline_depth 範囲内の場合は、Headline 化する
        print(depth_count)
        print("#" + target['name'])                # TODO:ヘッドライン処理
        
    before_depth = target['depth']                # 直前の深さを保持


1
#FolderA
2
#FolderA1
2
#FolderA2
2
#FolderA3
2
#FolderA4
1
#FolderB
2
#FolderB11


In [8]:
target_path_list

[{'depth': 2,
  'drive': '',
  'name': 'FolderA',
  'path': './test/FolderA/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA1',
  'path': './test/FolderA/FolderA1/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA2',
  'path': './test/FolderA/FolderA2/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA3',
  'path': './test/FolderA/FolderA3/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA4',
  'path': './test/FolderA/FolderA4/keyfile.txt'},
 {'depth': 2,
  'drive': '',
  'name': 'FolderB',
  'path': './test/FolderB/keyfile.txt'},
 {'depth': 4,
  'drive': '',
  'name': 'FolderB11',
  'path': './test/FolderB/FolderB1/FolderB11/keyfile.txt'}]

In [9]:
sorted(target_path_list, key=lambda x:x['depth'])

[{'depth': 2,
  'drive': '',
  'name': 'FolderA',
  'path': './test/FolderA/keyfile.txt'},
 {'depth': 2,
  'drive': '',
  'name': 'FolderB',
  'path': './test/FolderB/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA1',
  'path': './test/FolderA/FolderA1/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA2',
  'path': './test/FolderA/FolderA2/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA3',
  'path': './test/FolderA/FolderA3/keyfile.txt'},
 {'depth': 3,
  'drive': '',
  'name': 'FolderA4',
  'path': './test/FolderA/FolderA4/keyfile.txt'},
 {'depth': 4,
  'drive': '',
  'name': 'FolderB11',
  'path': './test/FolderB/FolderB1/FolderB11/keyfile.txt'}]

In [10]:
target_path_list[0]['network_drive']

KeyError: 'network_drive'

これらの関数の多くは Windows の一律命名規則 (UNCパス名) を正しくサポートしていません。 splitunc() と ismount() は正しく UNC パス名を操作できます。

os.path.splitunc(path)(原文)
パス名 path をペア (unc, rest) に分割します。ここで unc は (r'\\host\mount' のような) UNC マウントポイント、そして rest は (r'\path\file.ext' のような) パスの残りの部分です。ドライブ名を含むパスでは常に unc が空文字列になります。

利用可能: Windows

os.path.splitunc(path)
Deprecated since version 3.1: Use splitdrive instead.


In [ ]:
os.sep

In [ ]:
for path in target_path_list:
    print (os.path.splitdrive(path[0]))